<a href="https://colab.research.google.com/github/markus-hoanglong/test-gitpage/blob/master/instant_ngp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/ceres-solver-v2.zip
!unzip /content/ceres-solver-v2.zip -d ceres-solver

!cp -r /content/ceres-solver/lib/. /usr/local/lib
!chmod 755 /content/ceres-solver/bin/colmap
!cp -r /content/ceres-solver/bin/. /usr/local/bin

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/instant-ngp-v2.zip
!unzip /content/instant-ngp-v2.zip -d instant-ngp

!pip install commentjson huggingface-hub
%cd /content/instant-ngp


In [3]:
import os
from huggingface_hub import create_repo, upload_folder
from google.colab import drive

drive.mount('/content/gdrive')

!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/ceres-solver-v2.zip
!unzip /content/ceres-solver-v2.zip -d ceres-solver

!cp -r /content/ceres-solver/lib/. /usr/local/lib
!chmod 755 /content/ceres-solver/bin/colmap
!cp -r /content/ceres-solver/bin/. /usr/local/bin

!wget https://github.com/camenduru/instant-ngp-colab/releases/download/v1.0/instant-ngp-v2.zip
!unzip /content/instant-ngp-v2.zip -d instant-ngp

!pip install commentjson huggingface-hub
%cd /content/instant-ngp

scene_path = "/content/gdrive/MyDrive/fox" #@param {type:"string"}
train_path = "/content/train"
if not os.path.isdir(scene_path):
  raise NotADirectoryError(scene_path)
!rm -rf {train_path}
!mkdir {train_path}
!cp -r {scene_path}/. /content/train
transforms_path = os.path.join(train_path, f"transforms.json")
!python ./scripts/colmap2nerf.py --colmap_matcher exhaustive --run_colmap --aabb_scale 4 --images {train_path} --out {transforms_path}

train_steps = 2000  #@param {type:"integer"}
snapshot_path = os.path.join(train_path, f"{train_steps}.msgpack")
!python ./scripts/run.py --scene {train_path} --mode nerf --n_steps {train_steps} --save_snapshot {snapshot_path}

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libatlas-base-dev is already the newest version (3.10.3-12ubuntu1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
The following additional packages will be installed:
  at-spi2-core gsettings-desktop-schemas libamd2 libatk-bridge2.0-0
  libatk1.0-0 libatk1.0-data libatspi2.0-0 libboost-atomic1.74-dev
  libboost-atomic1.74.0 libboost-chrono1.74-dev libboost-chrono1.74.0
  libboost-date-time1.74-dev libboost-date-time1.74.0
  libboost-filesystem1.74-dev libboost-filesystem1.74.0 libboost-graph1.74-dev
  libboost-graph1.74.0 libboost-program-options1.74-dev
  libboost-program-options1.74.0 libboost-regex1.74-dev libboost-regex1.74.0
  libboost-serialization1.74-dev libboost-serialization1.74.0
  libboost-system1.74-dev libboost-system1.74.0 libboos

In [ ]:
hf_token = "" #@param {type:"string"}
repo_id = "camenduru/instant-ngp-models" #@param {type:"string"}
path_in_repo = "fox" #@param {type:"string"}
create_repo(repo_id, private=True, token=hf_token)
upload_folder(folder_path=train_path, path_in_repo=path_in_repo, repo_id=repo_id, commit_message=f"{scene_path}", token=hf_token)